In [ ]:
import requests
import time
from random import random
from bs4 import BeautifulSoup
import pandas as pd

data = []

def tiki_scraper(category_url, page_start=1):

    page = page_start

    while True:
        # generates random delay between 0.5 and 1.5 seconds:
        delay = random() + 0.5

        # defines target page url 
        current = category_url + str(page)

        # general url business and creating soup obj:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
        r = requests.get(current, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")
        
        # populates list of all products on target page:
        products = soup.find_all("a", {"class": "product-item"})

        #for debugging and general reassurance:
        print(f"Currently scraping {len(products)} products from page {page}.")
        
        #checks if there are actually products on the page (i.e. if it has come to final page):
        if len(products) != 0:
            
            for product in products:
                #creates dictionary of product info:
                d = {'product_id':'', 'free_shipping':'', 'title':'', 'price':'', 'image_url':'', 'reviewer_tot': ''}

                try:
                    d['product_id'] = (product['href'].split('-')[-1])[:-5] 
                except:
                    d['product_id'] = 0
                    print("product_id empty")
                try:
                    d['title'] = product.find('div', {'class':'name'}).text
                except:
                    d['title'] = 0
                    print("title empty")
                try:
                    d['price'] = product.find('div', {'class':'price-discount__price'}).text
                except:
                    d['price'] = 0
                    print("price empty")
                try:
                    d['image_url'] = product.img['src']
                except:
                    d['image_url'] = 0
                    print("image_url empty")
                try:
                    d['free_shipping'] = bool(product.find('div', {'class':'badge-top'}))
                except:
                    d['free_shipping'] = 0
                    print("free shipping empty")
                try:
                    d['reviewer_tot'] = (product.find('div', {'class':'review'}).text).strip('()')
                except:
                    d['reviewer_tot'] = 0
                    print("review_tot empty")

                data.append(d)
        else:
            break
        
        # adds 1 to page number to move loop forward
        page += 1

        #delays next loop initiation
        time.sleep(delay)


In [ ]:
tiki_scraper('https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner&page=',205)

In [ ]:
table = pd.DataFrame(data)